In [ ]:
%matplotlib widget

Jupyter Notebook example
========================

Creating a cube
---------------

This example will create a **500x500** pixels cube with the 12-bands images
from **S-PLUS TILE HYDRA-0045** for the *NGC3312* galaxy. The stamps are made
from a cropped 500x500 pixels area located at **S-PLUS TILE** mentioned before,
centered at coordinates RA *10h37m02.5s* and DEC *-27d33’56"*.

![NGC3312 crop at HYDRA-0045 S-PLUS tile](HYDRA-0045_NGC3312-stamp.png)

The ``scubes`` entry-point script will download the 12-band stamps
and calculate the fluxes and errors of each image. The images are 
zero-point calibrated based on the **S-PLUS iDR4** (*scubes package 
includes zp data*) values, but they are **not corrected for Galactic 
extinction**. The resultant files will be created at directory *workdir*.

S-PLUS filters information included:

In [ ]:
!scubes_filters --decimals 2

The program also could use SExtractor in order to create a spatial mask of
stars, attempting to remove the areas enclosed by the brightest ones
along the FOV (*-M* optional argument). To use this option, do not forget 
to include the SExtractor executable path using the option *-x*. An example
of this usage is at the end of this document.

``scubes`` entry-point script help and usage:

In [ ]:
!scubes --help

The call to the entry-point script ``scubes`` to this example would be:

*Do not forget to change YOURUSER and YOURPASS for your credentials at S-PLUS Cloud.*

In [ ]:
!scubes -frR -w . -U YOURUSER -P YOURPASS -l 500 -- HYDRA-0045 10h37m02.5s -27d33\'56\" NGC3312

How to read a cube
------------------

A cube resultant from ``scubes`` script is stored using FITS format 
and is called **SCUBE**. In order to help the *user*, ``scubes`` 
package implements a utility class to read the **SCUBE**. This class
implements the method ``lRGB_image()`` to create Lupton RGB images.

Below an example of the ``scubes.utilities.read_scube()`` usage:

In [ ]:
from os.path import join
from scubes.utilities.readscube import read_scube

SNAME = f'NGC3312'
filename = join(SNAME, f'{SNAME}_cube.fits')

scube = read_scube(filename)

In [ ]:
scube.primary_header

In [ ]:
scube.data_header

In [ ]:
scube.filters

In [ ]:
scube.metadata

In [ ]:
scube.weimask__yx.sum()

In [ ]:
xi, xf = 0, -1
yi, yf = 0, -1
xslice = slice(xi, xf)
yslice = slice(yi, yf)

Images plot
-----------

This example below plots the images of the magnitudes and errors from 
all filters, calculated by ``scubes`` package.

The unities are in mag/arcsec/pix$^2$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

f__byx = scube.mag__lyx[:, yslice, xslice]
ef__byx = scube.emag__lyx[:, yslice, xslice]
weimask__yx = scube.weimask__yx[yslice, xslice]
filters_names = scube.metadata['FILTER']
nb, ny, nx = f__byx.shape
nrows = 2
ncols = int(nb/nrows)
f, ax_arr = plt.subplots(2*nrows, ncols)
f.set_size_inches(12, 6)
f.subplots_adjust(left=0)
k = 0
for ir in range(nrows):
    for ic in range(ncols):
        mask = weimask__yx | (f__byx[k] <= 0) | ~(np.isfinite(ef__byx[k]))
        img = np.ma.masked_array(f__byx[k], mask=mask)
        #vmin, vmax = np.percentile(img.compressed(), [5, 95])
        vmin, vmax = 16, 25
        ax = ax_arr[ir*2, ic]
        ax.set_title(filters_names[k])
        im = ax.imshow(img, origin='lower', cmap='Spectral', vmin=vmin, vmax=vmax)
        plt.colorbar(im, ax=ax)
        ax.xaxis.set_major_locator(ticker.NullLocator())
        ax.yaxis.set_major_locator(ticker.NullLocator())

        eimg = np.ma.masked_array(ef__byx[k], mask=mask)
        #vmin, vmax = np.percentile(eimg.compressed(), [5, 95])
        vmin, vmax = 0, 0.5
        ax = ax_arr[ir*2 + 1, ic]
        ax.set_title(f'err {filters_names[k]}')
        im = ax.imshow(eimg, origin='lower', cmap='Spectral', vmin=vmin, vmax=vmax)
        plt.colorbar(im, ax=ax)
        ax.xaxis.set_major_locator(ticker.NullLocator())
        ax.yaxis.set_major_locator(ticker.NullLocator())

        k += 1

RGB and Filters plot
--------------------

This series of examples below are using the Lupton RGB method of the 
``scube`` module and some of the **data** stored by the ``scubes`` 
package in order to create a filters transmittance plot.

``scubes`` package implements various constants in ``scubes.constants``
module. This example below uses ``FILTER_NAMES_FITS``, ``FILTERS_COLORS`` 
and ``FILTER_TRANSMITTANCE`` data. The filters transmittance curves are 
calculated by ``splus-filters`` package:

* https://github.com/splus-collab/splus_filters

The ``scubes.utilities.read_scube.lRGB_image`` is a wrapper to 
``astropy.visualization.make_lupton_rgb()`` that creates RGB images 
using the **SCUBE** instantiated by ``scubes.utilities.read_scube``
class.

In [ ]:
# RGB plot

rgb_f = [1, 1, 1]
pminmax = [5, 95]
Q = 10
stretch = 5
im_max = 1
minimum = (0, 0, 0)

f, axArr = plt.subplots(1, 3)
ax1, ax2, ax3 = axArr
f.set_size_inches(12, 4)

###### original RGB from splus.data package
rgb = ['iSDSS', 'rSDSS', 'gSDSS']
rgb__yxb = scube.lRGB_image(rgb, rgb_f, pminmax, Q=Q, stretch=stretch, im_max=im_max, minimum=minimum)
ax1.imshow(rgb__yxb[yslice, xslice, :], origin='lower')
ax1.set_title(rgb)

###### improved RGB
R_filters = 'iSDSS'
G_filters = 'rSDSS'
B_filters = tuple(scube.filters[0:5])
rgb = [R_filters, G_filters, B_filters]
rgb__yxb = scube.lRGB_image(rgb, rgb_f, pminmax, Q=Q, stretch=stretch, im_max=im_max, minimum=minimum)
ax2.imshow(rgb__yxb[yslice, xslice, :], origin='lower')
ax2.set_title(f'B={B_filters}')

###### improved RGB with contrast
rgb_f = [1, 1, 2]
R_filters = 'iSDSS'
G_filters = 'rSDSS'
B_filters = tuple(scube.filters[0:5])
rgb = [R_filters, G_filters, B_filters]
rgb__yxb = scube.lRGB_image(rgb, rgb_f, pminmax, Q=Q, stretch=stretch, im_max=im_max, minimum=minimum)
ax3.imshow(rgb__yxb[yslice, xslice, :], origin='lower')
ax3.set_title('new_B=2B')

In [ ]:
from matplotlib.gridspec import GridSpec

from scubes.constants import FILTER_NAMES_FITS, FILTER_COLORS, FILTER_TRANSMITTANCE

# central coords
cent_pix = int(scube.size/2)
spec_pix_y, spec_pix_x = cent_pix, cent_pix

# data
logflux__l = np.ma.log10(scube.flux__lyx[:, cent_pix-30:cent_pix+30, cent_pix-30:cent_pix+30].sum(axis=(1, 2)))
bands__l = scube.pivot_wave

# plot
f = plt.figure()
f.set_size_inches(12, 5)
f.subplots_adjust(left=0, right=0.9)

nrows = 4
ncols = 2

gs = GridSpec(nrows=nrows, ncols=ncols, hspace=0, wspace=0.03, figure=f)

ax = f.add_subplot(gs[0:nrows - 1, 1])
axf = f.add_subplot(gs[-1, 1])
axrgb = f.add_subplot(gs[:, 0])

# RGB image
rgb = ['iSDSS', 'rSDSS', 'gSDSS']
rgb_f = [1, 1, 1]
pminmax = [5, 95]
Q = 10
stretch = 5
im_max = 1
minimum = (0, 0, 0)
rgb__yxb = scube.lRGB_image(rgb, rgb_f, pminmax, Q=Q, stretch=stretch, im_max=im_max, minimum=minimum)
axrgb.imshow(rgb__yxb[yslice, xslice, :], origin='lower')
axrgb.set_title('Lupton RGB')

# filters transmittance
filter_colors = []
axf.sharex(ax)
for k in scube.filters:
    _f = FILTER_TRANSMITTANCE[k]
    c = FILTER_COLORS[FILTER_NAMES_FITS[k]]
    filter_colors.append(c)
    lt = '--'
    if 'JAVA' in k or 'SDSS' in k:
        lt = '-'
    axf.plot(_f['wavelength'], _f['transmittance'], c=c, lw=1, ls=lt, label=k)
axf.legend(loc=(0.82, 1.15), frameon=False)

# spectrum 
ax.set_title(f'{scube.galaxy} @ {scube.tile} ({spec_pix_x},{spec_pix_y})')
ax.plot(bands__l, logflux__l, ':', c='k')
ax.scatter(bands__l, logflux__l, c=np.array(filter_colors), s=1)
ax.plot(bands__l, logflux__l, '-', c='lightgray')
ax.scatter(bands__l, logflux__l, c=np.array(filter_colors))

ax.set_xlabel(r'$\lambda_{\rm pivot}\ [\AA]$', fontsize=10)
ax.set_ylabel(r'$\log_{10}f\ [{\rm erg}\ \AA^{-1}{\rm s}^{-1}{\rm cm}^{-2}]$', fontsize=10)
axf.set_xlabel(r'$\lambda\ [\AA]$', fontsize=10)
axf.set_ylabel(r'${\rm R}_\lambda\ [\%]$', fontsize=10)

In [ ]:
# Contour plot example
image__yx = scube.mag__lyx[scube.filters.index('rSDSS'), yslice, xslice]

f = plt.figure()
im = plt.imshow(image__yx, cmap='Spectral_r', origin='lower', vmin=16, vmax=25)
plt.contour(image__yx, levels=[21, 23, 24], colors=['k', 'gray', 'lightgray'])
plt.colorbar(im)

Mask stars with SExtractor
--------------------------

``scubes`` package implements some ways to create a star mask for the selected
stamp. The program uses [source-extractor](https://www.astromatic.net/software/sextractor/)
to create the mask and depends on a series of dedicated options.

Before the **SCUBE** created, one can choose two ways to create a stars mask:

1. ``scubes`` script has the option to include the source extractor (--mask_stars option);
2. ``sex_mask_stars`` script: Another script implemented by ``scubes`` package which creates a stars mask either using RA, DEC, TILE and SIZE options.

After the **SCUBE** generated, one can use two other options to create a stars mask:
1. ``sex_mask_stars_cube`` script, or;
2. reading a cube with ``scubes.utilities.read_scube`` class and using the method ``source_extractor()``.

Here we show the most forward example to create a stars mask for us now, using the instantiated object ``scube``:

In [ ]:
scube.source_extractor(sextractor='source-extractor', username=YOURUSER, password=YOURPASS)

In [ ]:
scube.mask_stars_hdul.info()

In [ ]:
# get recent create stars mask hdulist
mask = scube.mask_stars_hdul['MASK'].data

f = plt.figure()
ax = f.gca()

r_image__yx = scube.mag__lyx[scube.filters.index('rSDSS')]
msk_r_image__yx = np.ma.masked_array(r_image__yx, mask=mask)
im = ax.imshow(msk_r_image__yx, origin='lower', cmap='Spectral', vmin=16, vmax=25)
f.colorbar(im)

 Now, the same example but using the ``sex_mask_stars_cube`` script:

In [ ]:
!sex_mask_stars_cube -U YOURUSER -P YOURPASS -N -F -x source_extractor NGC3312/NGC3312_cube.fits

In [ ]:
from astropy.io import fits

mask = fits.getdata('NGC3312_sexmask.fits', 'MASK')

f = plt.figure()
ax = f.gca()

r_image__yx = scube.mag__lyx[scube.filters.index('rSDSS')]
msk_r_image__yx = np.ma.masked_array(r_image__yx, mask=mask)
im = ax.imshow(msk_r_image__yx, origin='lower', cmap='Spectral', vmin=16, vmax=25)
f.colorbar(im)